In [12]:
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling

## Import raster file representing xxx

In [10]:
src = rasterio.open('processed_data/statewidehabitatmap.tiff')

In [11]:
src.crs

CRS.from_wkt('PROJCS["NAD_1983_Lambert_Conformal_Conic",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",41.75],PARAMETER["central_meridian",-120.5],PARAMETER["standard_parallel_1",43],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",1312335.95800525],PARAMETER["false_northing",0],UNIT["foot",0.3048,AUTHORITY["EPSG","9002"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [13]:
#from rasterio docs - https://rasterio.readthedocs.io/en/latest/topics/reproject.html

dst_crs = 'EPSG:4326'

with rasterio.open('processed_data/statewidehabitatmap.tiff') as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open('processed_data/statewidehabitatmap.wgs84.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

In [15]:
src = rasterio.open('processed_data/statewidehabitatmap.wgs84.tif')

In [16]:
src.crs

CRS.from_epsg(4326)

In [17]:
src.count

1

In [18]:
raster_arr = src.read(1)

In [19]:
raster_arr.shape

(13020, 25871)